<a href="https://colab.research.google.com/github/jimmyshah83/deep_learning/blob/master/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [0]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
import re
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx

# Loading Data

In [0]:
f = "C:/Users/334461373/Downloads/projectdata/projectdata.xlsx"

file = pd.read_excel(f)
data = file['new_strategy_extracted']


In [0]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [0]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [0]:
def generate_summary(article_text):
    stop_words = stopwords.words('english')
    summarize_text = []
    
    article = article_text.split(". ")
    sentences = []

    for sentence in article:
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(4):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    summary=  ". ".join(summarize_text)
    return summary;

In [0]:
# Actual Text 
print(data[0])

The Fund seeks to track the investment results of the MSCI Germany Index (the “Underlying Index”), which consists of stocks traded primarily on the Frankfurt Stock Exchange. The Underlying Index may include large-, mid- or small-capitalization companies. Components of the Underlying Index primarily include consumer discretionary, financials and healthcare companies. The components of the Underlying Index, and the degree to which these components represent certain industries, are likely to change over time.
BFA uses a “passive” or indexing approach to try to achieve the Fund’s investment objective. Unlike many investment companies, the Fund does not try to “beat” the index it tracks and does not seek temporary defensive positions when markets decline or appear overvalued.
Indexing may eliminate the chance that the Fund will substantially outperform the Underlying Index but also may reduce some of the risks of active management, such as poor security selection. Indexing seeks to achieve


In [0]:
# Summarized Text
Output = generate_summary(data[0])
print(Output)

The Fund may or may not hold all of the securities in an applicable underlying index.
The Fund will at all times invest at least 80% of its assets in the securities of its Underlying Index and in depositary receipts representing securities in its Underlying Index. The Fund seeks to track the investment results of the Underlying Index before fees and expenses of the Fund.
The Fund may lend securities representing up to one-third of the value of the Fund's total assets (including the value of any collateral received).
The Underlying Index is sponsored by MSCI Inc. The Index Provider determines the composition and relative weightings of the securities in the Underlying Index and publishes information regarding the market value of the Underlying Index.. “Representative sampling” is an indexing strategy that involves investing in a representative sample of securities that collectively has an investment profile similar to that of an applicable underlying index
